## Techniche - Topic Modelling

In [1]:
import pandas as pd
import numpy as np

import gensim
import gensim.corpora as corpora
from gensim.corpora import mmcorpus
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.phrases import Phrases, Phraser
from gensim.models.ldamodel import LdaModel
from gensim.models import AuthorTopicModel, atmodel
from gensim.test.utils import common_dictionary, datapath, temporary_file
from smart_open import smart_open

import spacy
from spacy.lemmatizer import Lemmatizer
from spacy.lang.en import LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, punkt, RegexpTokenizer, wordpunct_tokenize
from nltk.stem import PorterStemmer, LancasterStemmer, WordNetLemmatizer

import json
from pandas.io.json import json_normalize
import requests
import re
import os
import calendar

from topic_model import tokenize_docs#, (TODO) Lee convert_bytes

from smart_open import smart_open

import matplotlib.pyplot as plt
import pyLDAvis
import pyLDAvis.gensim

from pprint import pprint

import pyspark
import pyspark.sql.functions as F

/anaconda3/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
%load_ext autoreload

# pd.set_option('display.max_colwidth', -1)
pd.options.display.max_columns = 10
pd.set_option('display.max_rows', 10)

In [3]:
np.random.seed(3)

In [4]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

### Import Data

#### Import data from PatentsView API

In [5]:
# patents endpoint
endpoint_url = 'http://www.patentsview.org/api/patents/query'

# build list of possible fields that endpoint request will return
df = pd.read_excel("data/patents_view_patents_fields.xlsx")
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_').str.replace('(', '').str.replace(')', '')
pat_fields = df.api_field_name.values.tolist()

#### Import initial dataset

In [6]:
# build query - initial small dataset
query={"_or":[{"_text_phrase":{"patent_title":"natural language"}},{"_text_phrase":{"patent_abstract":"natural language"}}]}
# uncomment to use alternate query options
# query={"cpc_subgroup_id":"G06T3/4046"}
# query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
# query={"_and":
#         [{"_or":
#             [{"_text_phrase":{"patent_title":"natural language"}}
#             ,{"_text_phrase":{"patent_abstract":"natural language"}}]}
#         ,{"_and":
#       [{"patent_year":2016}]}]} 
# query = {"_and":[{"_gte":{"patent_date":"2017-01-01"}},{"_lte":{"patent_date":"2017-01-31"}}]}
fields=pat_fields
options={"per_page":2500}
sort=[{"patent_date":"desc"}]

params={'q': json.dumps(query),
        'f': json.dumps(fields),
        'o': json.dumps(options),
        's': json.dumps(sort)}

# request and results
response = requests.get(endpoint_url, params=params)
status = response.status_code
print("status:", status)
results = response.json()
count = results.get("count")
total_pats = results.get("total_patent_count")
print("patents on current page:",count,';', "total patents:",total_pats)

status: 200
patents on current page: 2482 ; total patents: 2482


#### Structure data

In [7]:
# extract metadata from response
print("status code:", response.status_code,';', "reason:", response.reason)
total_patent_count = results["total_patent_count"]
patents_per_page = results['count']
print("total_patent_count:",total_patent_count,';', "patents_per_page:", patents_per_page)

# extract data from response
data_resp = results['patents']
# data_resp[0]

raw_df = pd.DataFrame(data_resp)
raw_df.head(3)

status code: 200 ; reason: OK
total_patent_count: 2482 ; patents_per_page: 2482


,IPCs,application_citations,applications,assignees,cited_patents,...,patent_year,pct_data,rawinventors,uspcs,wipos
0,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020077823', 'ap...","[{'app_country': 'US', 'app_date': '2013-07-26...","[{'assignee_city': 'Burlington', 'assignee_cou...",[{'cited_patent_category': 'cited by examiner'...,...,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Jeffrey N.', 'raw...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
1,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2002/20020138265', 'ap...","[{'app_country': 'US', 'app_date': '2017-09-11...","[{'assignee_city': 'Mountain View', 'assignee_...",[{'cited_patent_category': 'cited by applicant...,...,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Evgeny A.', 'rawi...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."
2,"[{'ipc_action_date': '2019-03-12', 'ipc_class'...","[{'appcit_app_number': '2001/20010029455', 'ap...","[{'app_country': 'US', 'app_date': '2016-09-28...","[{'assignee_city': 'Seattle', 'assignee_countr...",[{'cited_patent_category': 'cited by applicant...,...,2019,"[{'pct_102_date': None, 'pct_371_date': None, ...","[{'rawinventor_first_name': 'Thibault Pierre',...","[{'uspc_first_seen_date': None, 'uspc_last_see...","[{'wipo_field_id': None, 'wipo_field_title': N..."


#### Subset dataframe

In [8]:
# subset dataframe - comment/uncomment to include fields
df = raw_df[['patent_number', 
         'patent_date', 
         'patent_title',
         'patent_abstract', 
         'patent_firstnamed_assignee_id',
         'patent_firstnamed_assignee_location_id',
         'patent_firstnamed_assignee_latitude',
         'patent_firstnamed_assignee_longitude',
         'patent_firstnamed_assignee_city',
         'patent_firstnamed_assignee_state',
         'patent_firstnamed_assignee_country', 
         'patent_firstnamed_inventor_id',
         'patent_firstnamed_inventor_location_id',
         'patent_firstnamed_inventor_latitude',
         'patent_firstnamed_inventor_longitude',
         'patent_firstnamed_inventor_city',
         'patent_firstnamed_inventor_state',
         'patent_firstnamed_inventor_country',
         'patent_year', 
         'patent_type', 
         'patent_kind',
         'inventors'
            ]]
df.head(3)

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,...,patent_firstnamed_inventor_country,patent_year,patent_type,patent_kind,inventors
0,10229106,2019-03-12,Initializing a workspace for building a natura...,Designing a natural language understanding (NL...,org_ID497r4tFbCIaMBjGAST,...,US,2019,utility,B2,"[{'inventor_city': 'Newton', 'inventor_country..."
1,10229109,2019-03-12,Allowing spelling of arbitrary words,"Methods, systems, and apparatus, including com...",org_p6ofWD2xFNSnyYkj6wpA,...,CH,2019,utility,B1,"[{'inventor_city': 'Adliswil', 'inventor_count..."
2,10229113,2019-03-12,Leveraging content dimensions during the trans...,A content management system (CMS) and a transl...,org_Vbc6obpnxWM42d0HjlXY,...,US,2019,utility,B1,"[{'inventor_city': 'Seattle', 'inventor_countr..."


#### Explore data

In [9]:
# 561 different assignees
len(df.patent_firstnamed_assignee_id.unique())

561

#### Create new column

In [10]:
# create new column that combines the patent title and the patent abstract columns into a single string
df['patent_title_abstract'] = df.patent_title + ' ' + df.patent_abstract
df.patent_title_abstract.head(3)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


0    Initializing a workspace for building a natura...
1    Allowing spelling of arbitrary words Methods, ...
2    Leveraging content dimensions during the trans...
Name: patent_title_abstract, dtype: object

In [11]:
df.sort_values(by=['patent_date'])

,patent_number,patent_date,patent_title,patent_abstract,patent_firstnamed_assignee_id,...,patent_year,patent_type,patent_kind,inventors,patent_title_abstract
2481,3980994,1976-09-14,Text editing and display system having text in...,A natural language text editing and display sy...,org_1UVZxxNbuUPJkuDfvvpa,...,1976,utility,A,"[{'inventor_city': 'Andover', 'inventor_countr...",Text editing and display system having text in...
2480,4057849,1977-11-08,Text editing and display system,A text-editing and display system for editing ...,org_1UVZxxNbuUPJkuDfvvpa,...,1977,utility,A,"[{'inventor_city': 'Andover', 'inventor_countr...",Text editing and display system A text-editing...
2479,4502128,1985-02-26,Translation between natural languages,An input sentence described by a first natural...,org_70D1lR89kQnFiCFdJ6s5,...,1985,utility,A,"[{'inventor_city': 'Yokohama', 'inventor_count...",Translation between natural languages An input...
2478,4586160,1986-04-29,Method and apparatus for analyzing the syntact...,An automatic syntax analyzing method is applie...,org_hDziASDpeFilN1JsnVK0,...,1986,utility,A,"[{'inventor_city': 'Yokohama', 'inventor_count...",Method and apparatus for analyzing the syntact...
2477,4599612,1986-07-08,Displaying and correcting method for machine t...,In a system wherein a first text in a first na...,org_70D1lR89kQnFiCFdJ6s5,...,1986,utility,A,"[{'inventor_city': 'Fujisawa', 'inventor_count...",Displaying and correcting method for machine t...
...,...,...,...,...,...,...,...,...,...,...,...
11,10229687,2019-03-12,Scalable endpoint-dependent natural language u...,A computer-implemented technique is described ...,org_EilEWQcC6UiqHcSGx9mb,...,2019,utility,B2,"[{'inventor_city': 'Sammamish', 'inventor_coun...",Scalable endpoint-dependent natural language u...
12,10230677,2019-03-12,Identifying an entity associated with an onlin...,An approach is described for identifying an en...,org_q9Bn28RHhpYrQjKvraAH,...,2019,utility,B2,"[{'inventor_city': 'Sewickley', 'inventor_coun...",Identifying an entity associated with an onlin...
13,10230680,2019-03-12,Intelligently splitting text in messages poste...,"A method, system and computer program product ...",org_q9Bn28RHhpYrQjKvraAH,...,2019,utility,B2,"[{'inventor_city': 'Shanghai', 'inventor_count...",Intelligently splitting text in messages poste...
7,10229673,2019-03-12,System and method for providing follow-up resp...,"In certain implementations, follow-up response...",org_9D8x1qL3IRASp6GG7Glu,...,2019,utility,B2,"[{'inventor_city': 'Bellevue', 'inventor_count...",System and method for providing follow-up resp...


In [67]:
text_data = df.patent_title_abstract.tolist()
# text_data

In [13]:
# partition data
len(text_data)
text_train = text_data[:round(len(text_data)*.8)]
text_test = text_data[round(len(text_data)*.8):]
print(len(text_data), len(text_train), len(text_test), len(text_train)+len(text_test))

2482 1986 496 2482


### Pre-process text data

In [14]:
# uncomment to download stop words from nltk and language package from spacy
# nltk.download('stopwords')
# nltk.download('punkt')
# !python -m spacy download en

#### Tokenize

In [17]:
# tokenize documents

def tokenize_docs(docs):
    tokenized_docs = []
    for doc in docs:
        tokenized_docs.append(word_tokenize(doc))
    return tokenized_docs

tokenized_docs = tokenize_docs(text_train)

#### Clean punctuation

In [18]:
# clean punctuation
def clean_docs(tokenized_docs):
    clean_docs = []
    for doc in tokenized_docs:
       clean_docs.append([word for word in doc if word.isalpha()])  
    return clean_docs

In [19]:
cleaned_data = clean_docs(tokenized_docs)
cleaned_data[0]

['Initializing',
 'a',
 'workspace',
 'for',
 'building',
 'a',
 'natural',
 'language',
 'understanding',
 'system',
 'Designing',
 'a',
 'natural',
 'language',
 'understanding',
 'NLU',
 'model',
 'for',
 'an',
 'application',
 'from',
 'scratch',
 'can',
 'be',
 'difficult',
 'for',
 'A',
 'system',
 'can',
 'simplify',
 'the',
 'design',
 'process',
 'by',
 'providing',
 'an',
 'interface',
 'allowing',
 'a',
 'designer',
 'to',
 'input',
 'example',
 'usage',
 'sentences',
 'and',
 'build',
 'an',
 'NLU',
 'model',
 'based',
 'on',
 'presented',
 'matches',
 'to',
 'those',
 'example',
 'sentences',
 'In',
 'one',
 'embodiment',
 'a',
 'method',
 'for',
 'initializing',
 'a',
 'workspace',
 'for',
 'building',
 'an',
 'NLU',
 'system',
 'includes',
 'parsing',
 'a',
 'sample',
 'sentence',
 'to',
 'select',
 'at',
 'least',
 'one',
 'candidate',
 'stub',
 'grammar',
 'from',
 'among',
 'multiple',
 'candidate',
 'stub',
 'grammars',
 'The',
 'method',
 'can',
 'include',
 'presen

#### Convert to lowercase

In [20]:
# convert to lowercase
def lower_words(docs):
    lowered_words = []
    for doc in docs:
        lowered_words.append([word.lower() for word in doc])
    return lowered_words

lowered_data = lower_words(cleaned_data)
lowered_data[0]

['initializing',
 'a',
 'workspace',
 'for',
 'building',
 'a',
 'natural',
 'language',
 'understanding',
 'system',
 'designing',
 'a',
 'natural',
 'language',
 'understanding',
 'nlu',
 'model',
 'for',
 'an',
 'application',
 'from',
 'scratch',
 'can',
 'be',
 'difficult',
 'for',
 'a',
 'system',
 'can',
 'simplify',
 'the',
 'design',
 'process',
 'by',
 'providing',
 'an',
 'interface',
 'allowing',
 'a',
 'designer',
 'to',
 'input',
 'example',
 'usage',
 'sentences',
 'and',
 'build',
 'an',
 'nlu',
 'model',
 'based',
 'on',
 'presented',
 'matches',
 'to',
 'those',
 'example',
 'sentences',
 'in',
 'one',
 'embodiment',
 'a',
 'method',
 'for',
 'initializing',
 'a',
 'workspace',
 'for',
 'building',
 'an',
 'nlu',
 'system',
 'includes',
 'parsing',
 'a',
 'sample',
 'sentence',
 'to',
 'select',
 'at',
 'least',
 'one',
 'candidate',
 'stub',
 'grammar',
 'from',
 'among',
 'multiple',
 'candidate',
 'stub',
 'grammars',
 'the',
 'method',
 'can',
 'include',
 'presen

#### Clean stopwords

In [21]:
# clean stopwords

stop_words = stopwords.words('english')

In [22]:
def filter_stopwords(docs):
    filtered_docs = []
    for doc in docs:
       filtered_docs.append([word for word in doc if word not in stop_words])
    return filtered_docs

# remove stopwords
filtered_data = filter_stopwords(lowered_data)
filtered_data
# TODO (Lee) - resolve un-lowered stopwords "A" and "An", 'By', 'The'

[['initializing',
  'workspace',
  'building',
  'natural',
  'language',
  'understanding',
  'system',
  'designing',
  'natural',
  'language',
  'understanding',
  'nlu',
  'model',
  'application',
  'scratch',
  'difficult',
  'system',
  'simplify',
  'design',
  'process',
  'providing',
  'interface',
  'allowing',
  'designer',
  'input',
  'example',
  'usage',
  'sentences',
  'build',
  'nlu',
  'model',
  'based',
  'presented',
  'matches',
  'example',
  'sentences',
  'one',
  'embodiment',
  'method',
  'initializing',
  'workspace',
  'building',
  'nlu',
  'system',
  'includes',
  'parsing',
  'sample',
  'sentence',
  'select',
  'least',
  'one',
  'candidate',
  'stub',
  'grammar',
  'among',
  'multiple',
  'candidate',
  'stub',
  'grammars',
  'method',
  'include',
  'presenting',
  'user',
  'respective',
  'representations',
  'candidate',
  'stub',
  'grammars',
  'selected',
  'parsing',
  'sample',
  'sentence',
  'method',
  'include',
  'enabling',
 

#### Construct bigrams and trigrams

In [23]:
# train bigram phrases model
bigram_model = Phrases(filtered_data, min_count=1, threshold=1)

# train trigram phrases model
trigram_model = Phrases(bigram_model[filtered_data], threshold=100)  

/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [24]:
# bigrams
def bigrams(docs):
    """create bigrams"""
    return [bigram_model[doc] for doc in docs]

In [25]:
# initialize bigram and trigram models
bigram_model = gensim.models.phrases.Phraser(bigram_model)
trigram_model = gensim.models.phrases.Phraser(trigram_model)

In [26]:
bigrams(filtered_data)[0]

['initializing_workspace',
 'building_natural',
 'language_understanding',
 'system_designing',
 'natural_language',
 'understanding_nlu',
 'model',
 'application',
 'scratch',
 'difficult',
 'system',
 'simplify',
 'design_process',
 'providing_interface',
 'allowing',
 'designer',
 'input',
 'example',
 'usage',
 'sentences_build',
 'nlu_model',
 'based',
 'presented',
 'matches',
 'example',
 'sentences',
 'one_embodiment',
 'method',
 'initializing_workspace',
 'building',
 'nlu',
 'system_includes',
 'parsing_sample',
 'sentence',
 'select',
 'least_one',
 'candidate_stub',
 'grammar',
 'among_multiple',
 'candidate_stub',
 'grammars',
 'method_include',
 'presenting',
 'user',
 'respective_representations',
 'candidate_stub',
 'grammars',
 'selected',
 'parsing_sample',
 'sentence',
 'method_include',
 'enabling_user',
 'choose',
 'one',
 'respective_representations',
 'candidate_stub',
 'grammars',
 'method_include',
 'adding',
 'workspace',
 'stub_grammar',
 'corresponding',
 '

In [ ]:
# def trigrams(docs):
#     """create trigrams"""
#     return [trigram_model[bigram_model[doc]] for doc in docs]

In [ ]:
# trigrams(filtered_data)[0]

#### Stem and Lemmatize

In [ ]:
# def lemmatize_docs(docs, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
#     """lemmatize documents"""
#     lemmatized_docs = []
#     for doc in docs: 
#         lemmatized_docs.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
#     return lemmatized_docs

In [ ]:
# # TODO (Lee)
# # TODO (Lee) - lemmatize_docs(cleaned_data)

# lemmatizer = Lemmatizer(LEMMA_INDEX, LEMMA_EXC, LEMMA_RULES)
# # for doc in cleaned_data:
# #     for token in doc:
# #         token.lemma_

# # uncomment to use
# # download english model with "python -m spacy download en"

# # for token in doc:
# #     print(token, token.lemma, token.lemma_)



In [99]:
len(texts)

1986

#### Create corpus and dictionary

In [27]:
 # build dictionary
id_to_word = corpora.Dictionary(filtered_data)

# build corpus
texts = filtered_data

# apply term document frequency
# converts documents in corpus to bag-of-words format, a list of (token_id, token_count) tuples
corpus = [id_to_word.doc2bow(text) for text in texts]

In [28]:
# view formatted corpus (term-doc-frequency)
[[(id_to_word[id], freq) for id, freq in text] for text in corpus][:1]

[[('adding', 1),
  ('allowing', 1),
  ('among', 1),
  ('application', 1),
  ('based', 1),
  ('build', 1),
  ('building', 2),
  ('candidate', 5),
  ('choose', 1),
  ('chosen', 1),
  ('corresponding', 1),
  ('design', 1),
  ('designer', 1),
  ('designing', 1),
  ('difficult', 1),
  ('embodiment', 1),
  ('enabling', 1),
  ('example', 2),
  ('grammar', 3),
  ('grammars', 3),
  ('include', 3),
  ('includes', 1),
  ('initializing', 2),
  ('input', 1),
  ('interface', 1),
  ('language', 2),
  ('least', 1),
  ('matches', 1),
  ('method', 4),
  ('model', 2),
  ('multiple', 1),
  ('natural', 2),
  ('nlu', 3),
  ('one', 3),
  ('parsing', 2),
  ('presented', 1),
  ('presenting', 1),
  ('process', 1),
  ('providing', 1),
  ('representation', 1),
  ('representations', 2),
  ('respective', 2),
  ('sample', 2),
  ('scratch', 1),
  ('select', 1),
  ('selected', 1),
  ('sentence', 2),
  ('sentences', 2),
  ('simplify', 1),
  ('stub', 6),
  ('system', 3),
  ('understanding', 2),
  ('usage', 1),
  ('user'

### Model - model #1

In [29]:
# TODO (Lee) - deprecation warnings
# construct LDA model
model_lda = LdaModel(corpus=corpus,
                     id2word=id_to_word,
                     num_topics=25, 
                     random_state=100,
                     update_every=1,
                     chunksize=100,
                     passes=10,
                     alpha='auto',
                     per_word_topics=True)

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

...

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

In [34]:
# print keywords in n topics
pprint(model_lda.print_topics())

[(0,
  '0.075*"question" + 0.071*"agent" + 0.068*"system" + 0.066*"questions" + '
  '0.034*"automated" + 0.024*"areas" + 0.024*"customer" + 0.020*"handling" + '
  '0.018*"highly" + 0.018*"utilizes"'),
 (1,
  '0.050*"product" + 0.037*"program" + 0.036*"computer" + 0.034*"topic" + '
  '0.027*"group" + 0.026*"displayed" + 0.022*"category" + '
  '0.022*"informational" + 0.021*"topics" + 0.020*"assigned"'),
 (13,
  '0.084*"processor" + 0.069*"answer" + 0.056*"answers" + 0.053*"entity" + '
  '0.033*"memory" + 0.029*"abstract" + 0.028*"universal" + 0.026*"executed" + '
  '0.025*"question" + 0.022*"analyze"'),
 (11,
  '0.059*"tokens" + 0.055*"containing" + 0.041*"token" + 0.028*"intelligent" + '
  '0.027*"entry" + 0.020*"comparing" + 0.020*"validation" + 0.018*"calendar" + '
  '0.017*"finite" + 0.017*"classes"'),
 (17,
  '0.212*"first" + 0.158*"second" + 0.038*"format" + 0.035*"color" + '
  '0.023*"character" + 0.022*"description" + 0.016*"parameter" + '
  '0.015*"solution" + 0.014*"variable" 

In [35]:
# print top 10 keywords that comprise topic with index of 0
pprint(model_lda.print_topic(24))
# the most import keywords, and the respective weight, that form topic 0 are

('0.056*"language" + 0.049*"system" + 0.045*"systems" + 0.043*"methods" + '
 '0.038*"natural" + 0.032*"computer" + 0.019*"translation" + 0.018*"disclosed" '
 '+ 0.017*"source" + 0.016*"output"')


In [36]:
# print top 10 keywords that comprise topic with index of 1
pprint(model_lda.print_topic(1))

('0.050*"product" + 0.037*"program" + 0.036*"computer" + 0.034*"topic" + '
 '0.027*"group" + 0.026*"displayed" + 0.022*"category" + 0.022*"informational" '
 '+ 0.021*"topics" + 0.020*"assigned"')


In [37]:
# TODO (Lee) - infer topic from keywords?

### Evaluate - model #1

In [38]:
# calculate perplexity metrics
perplexity = model_lda.log_perplexity(corpus)
perplexity

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/anaconda3/lib/python3.6/site-packages/gensim/models/ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id

-6.844332107681411

In [39]:
# TODO (Lee) - confirm that filtered_data is indeed the correct dataset to pass to texts param
# calculate coherence metric
coherence = CoherenceModel(model=model_lda, texts=filtered_data, dictionary=id_to_word, coherence='c_v')
coherence_1 = coherence.get_coherence()
coherence_1

0.3897699923252669

In [40]:
# calculate coherence metric or each of the n topicss
coherence_1 = coherence.get_coherence_per_topic()
coherence_1

[0.32348039739352397,
 0.35170360224714814,
 0.3950224423159941,
 0.2848252291115782,
 0.20291289135865515,
 0.3027047918616517,
 0.4265237682505485,
 0.43851529187478955,
 0.3476302468434924,
 0.4859671593015696,
 0.4664475299209598,
 0.5590420325628574,
 0.3950477611470393,
 0.4527489579742296,
 0.2517553388816768,
 0.39171188223083825,
 0.3201914839227447,
 0.546608689087153,
 0.32377826911718693,
 0.5323612347470144,
 0.41744640101842095,
 0.4466727393652505,
 0.404578560346497,
 0.355147469212505,
 0.3214256380383471]

In [41]:
# explore topics
pyLDAvis.enable_notebook()
viz_topics_1 = pyLDAvis.gensim.prepare(model_lda, corpus, id_to_word)
viz_topics_1
# TODO (Lee) - salient vs relevant terms in pyLDA ?

/anaconda3/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=True'.

To retain the current behavior and silence the warning, pass sort=False

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
22    -0.296610  0.095295       1        1  19.616728
24    -0.257584  0.024486       2        1   9.040085
2     -0.242337  0.002436       3        1   8.181612
19    -0.226222 -0.157323       4        1   6.641489
4     -0.167748  0.128771       5        1   5.828878
...         ...       ...     ...      ...        ...
17     0.118481  0.016679      21        1   2.102518
1      0.097846  0.011719      22        1   1.911783
11     0.134421  0.012914      23        1   1.582533
0      0.000283 -0.178883      24        1   1.573721
13     0.098293 -0.078338      25        1   1.559301

[25 rows x 5 columns], topic_info=     Category         Freq         Term        Total  loglift  logprob
term                                                                  
263   Default  1756.000000         data  1756.000000  30.0000  30.0000
33    Default  1763.000000          one  1763.000000  29.0000  29.0000
366   Default  1216.000000         text  1216.000000  28.0000  28.0000
164   Default  1177.000000        query  1177.000000  27.0000  27.0000
23    Default  1117.000000        input  1117.000000  26.0000  26.0000
...       ...          ...          ...          ...      ...      ...
1509  Topic25    24.000029   retrieving    46.716225   3.4949  -4.5965
2433  Topic25    17.694983         verb    25.342592   3.8017  -4.9013
50    Topic25    25.274797       system  3020.315430  -0.6224  -4.5447
60    Topic25    18.829226  determining   289.246307   1.4291  -4.8391
424   Topic25    17.895395   configured   139.424271   2.1080  -4.8900

[1071 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
2473     25  0.986703  abstract
580       2  0.503195    access
580      11  0.225114    access
580      13  0.258218    access
2440     11  0.982435  accessed
...     ...       ...       ...
82        2  0.069769     words
82        5  0.926534     words
3753     16  0.963670   working
2243     10  0.968659   written
2749     16  0.943115         x

[1440 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[23, 25, 3, 20, 5, 15, 9, 17, 22, 4, 16, 11, 13, 24, 7, 19, 10, 8, 6, 21, 18, 2, 12, 1, 14])

### Model 2-  Mallet model

In [45]:
# uncomment to download Mallet topic model
# !wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
# update this path
path_mallet = 'data/mallet-2.0.8/bin/mallet'

In [46]:
model_2 = gensim.models.wrappers.LdaMallet(path_mallet, corpus=corpus, num_topics=25, id2word=id_to_word)

In [47]:
# topics
pprint(model_2.show_topics(formatted=False))

[(14,
  [('interface', 0.0904320447787301),
   ('voice', 0.0727654364177016),
   ('environment', 0.04093055798495714),
   ('system', 0.04040580724156026),
   ('commands', 0.039706139583697744),
   ('process', 0.0376071366101102),
   ('command', 0.03620780129438517),
   ('devices', 0.031659961518278816),
   ('control', 0.023089032709463005),
   ('services', 0.021864614308203602)]),
 (21,
  [('method', 0.10574127906976744),
   ('generating', 0.07485465116279069),
   ('sentence', 0.07267441860465117),
   ('generated', 0.056322674418604654),
   ('include', 0.052688953488372096),
   ('rules', 0.05232558139534884),
   ('sentences', 0.03688226744186047),
   ('rule', 0.031068313953488372),
   ('generation', 0.026344476744186048),
   ('includes', 0.025799418604651164)]),
 (1,
  [('message', 0.0717217119405902),
   ('response', 0.05511041625952707),
   ('communication', 0.04318936877076412),
   ('processing', 0.042603087746726595),
   ('audio', 0.03400429939417628),
   ('messages', 0.02403752198

In [48]:
# calculate coherence metric
coherence_model_2 = CoherenceModel(model=model_2, texts=filtered_data, dictionary=id_to_word, coherence='c_v')
coherence_model_2 = coherence_model_2.get_coherence()
coherence_model_2

0.3419043998299942

### Model 3 - Author topic model

In [68]:
# construct inventor-to-doc mapping as df from nested inventors column in json api response
df_inventors = json_normalize(results['patents'], record_path=['inventors'], meta=['patent_number', 'patent_date'])
df_inventors = df_inventors[['inventor_id', 'patent_number', 'patent_date']]
df_inventors.sort_values(by=['patent_date'])
df_inventors.pop("patent_date")
df_inventors.head(3)

,inventor_id,patent_number
0,7788103-1,10229106
1,8352247-1,10229109
2,8515750-2,10229109


In [69]:
# TODO (Lee) - resolve workaround
df_idx = df
df_idx['idx'] = df.index
df_idx
df_idx_1 = df_idx[['patent_number', 'idx', 'inventors']]
df_idx_2 = df_idx_1.set_index('patent_number')
df_idx_2.pop('inventors')
df_idx_2
df_pat_idx = df_idx_2.T.to_dict('records')
# for i in df_pat_idx:
#     df_pat_idx = dict(i)
# df_pat_idx

# df_pat_idx = df_idx_2.T.to_dict('records')
# for i in df_pat_idx:
#     df_pat_idx = dict(i)
# df_pat_idx

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [70]:
df_inventors.set_index('inventor_id').T.to_dict('list')

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


{'7788103-1': ['8903712'],
 '8352247-1': ['9514743'],
 '8515750-2': ['9971758'],
 '8849675-1': ['9971758'],
 '9514743-1': ['9514743'],
 '9971758-5': ['9971758'],
 '10223356-1': ['10223356'],
 '10223356-2': ['10223356'],
 '10223356-3': ['10223356'],
 '10223356-5': ['10223356'],
 '9177341-1': ['10223356'],
 '9959271-8': ['10223356'],
 '8281187-1': ['10095736'],
 '9442919-1': ['10002124'],
 '9442919-2': ['9996526'],
 '9442919-3': ['10002124'],
 '9442919-5': ['9996526'],
 '8560468-5': ['9589060'],
 '9037568-4': ['9589060'],
 '9292545-3': ['9589060'],
 '9589060-4': ['9589060'],
 '9589060-5': ['9589060'],
 '9589060-6': ['9589060'],
 '10229187-1': ['10229189'],
 '10229187-2': ['10229189'],
 '6718519-1': ['10229189'],
 '7945556-2': ['10229189'],
 '9280340-1': ['10146858'],
 '7398209-3': ['7398209'],
 '8493600-1': ['10229678'],
 '6804574-3': ['9613020'],
 '9319838-3': ['9613020'],
 '9606701-1': ['9613020'],
 '10229680-1': ['10229680'],
 '10229680-2': ['10229680'],
 '5712675-3': ['10229687'],
 '

In [71]:
dict_pat2inv =df_inventors.set_index('patent_number').T.to_dict('list')
# dict_pat2inv

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: DataFrame columns are not unique, some columns will be omitted.
  """Entry point for launching an IPython kernel.


In [53]:
# for k, v in pat2inv.items():
#     name_dict[new_key] = name_dict.pop(k)
#     time.sleep(4)

# pprint.pprint(name_dict)

# d = {'x':1, 'y':2, 'z':3}
# d1 = {'x':'a', 'y':'b', 'z':'c'}

# dict((d1[key], value) for (key, value) in d.items())
# {'a': 1, 'b': 2, 'c': 3}

# idx_pat_map = df.patent_number.to_dict()
# idx_pat_map = {str(key): value for key, value in idx_pat_map.items()}
# import itertools
# x = list(itertools.islice(idx_pat_map.items(), 0, 4))
# x[:4]

In [66]:
pat2inv_dict = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}

In [72]:
# {k: list(v) for k,v in df_pat2inv.groupby("patent_number")["inventor_id"]}

In [73]:
df2 = df_inventors.groupby("patent_number")["inventor_id"]

In [74]:
df3 = df_idx_pat_inv_map.groupby("patent_number")["inventor_id"]

NameError: name 'df_idx_pat_inv_map' is not defined

In [79]:
pat2inv = {k: list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
len(pat2inv.items())

2482

In [92]:
pat2inv.items()
type(next(iter(pat2inv)))

str

In [81]:
pat2inv_2 = {str(k): list(v) for k,v in df_inventors.groupby("patent_number")["inventor_id"]}
len(pat2inv_2)

2482

In [85]:
# patdf2inv = dict((df_pat_idx[key], value) for (key, value) in dict_pat2inv.items())

In [95]:
type(corpus)

list

In [97]:
len(corpus)

1986

#### Construct author-topic model

In [94]:
# construct author-topic model
model_at = AuthorTopicModel(corpus=corpus,
                         doc2author=pat2inv,
                         id2word=id_to_word, 
                         num_topics=25)

TypeError: must be str, not int

In [ ]:
# construct vectors for authors
author_vecs = [model_at.get_author_topics(author) for author in model_at.id2author.values()]
author_vecs

In [93]:
# inspect topic distribution for author with id# 7788103-1
# each topic has a probability of being expressed given the particular inventor, but only the ones above a certain threshold are shown.

model_at['7788103-1']

NameError: name 'model_at' is not defined

In [ ]:
# def show_author(name):
#     print('\n%s' % name)
#     print('Docs:', model.author2doc[name])
#     print('Topics:')
#     pprint([(topic_labels[topic[0]], topic[1]) for topic in model[name]])

In [ ]:
# calculate per-word bound, which is a measure of the model's predictive performance (reconstruction error?)

build doc2author dictionary

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:

doc2author = atmodel.construct_doc2author(model.corpus, model.author2doc)

In [ ]:
gensim.models.atmodel.construct_author2doc(doc2author)
# construct mapping from author IDs to document IDs

Parameters:	doc2author (dict of (int, list of str)) – Mapping of document id to authors.
Returns:	Mapping of authors to document ids.
Return type:	dict of (str, list of int)

In [ ]:
gensim.models.atmodel.construct_doc2author(corpus, author2doc)
construct mapping from document IDs to author IDs

Parameters:	
corpus (iterable of list of (int, float)) – Corpus in BoW format.
author2doc (dict of (str, list of int)) – Mapping of authors to documents.
Returns:	
Document to Author mapping.

Return type:	
dict of (int, list of str)